Citibike data for Tableau
Module 18 Homework
Chris Gruenhagen

Data Plan:
 - download JC-YYYYMM-citibike-tripdata.csv.zip files from 2015-09 to 2022-12
 - read csvs from 2015-09 to 2021-01 into one pandas DataFrame (csvs with dataformat1)
 - read csvs from 2021-02 into a second pandas DataFrame (csvs with dataformat2)

Idea: loop
    - create lists
        * ldata1
        * ldata2
    - open csv into df
    - if csv[0]header == 'Trip Duration', append to ldata1
    - elif csv[0]header == 'Ride ID', append to ldata2
    end loop
    then loop through list1 and concatenate into bigdf1, loop through list2 and concatenate into bigdf2


dataformat1
    Trip Duration (seconds)
    Start Time and Date
    Stop Time and Date
    Start Station Name
    End Station Name
    Station ID
    Station Lat/Long
    Bike ID
    User Type (Customer = 24-hour pass or 3-day pass user; Subscriber = Annual Member)
    Gender (Zero=unknown; 1=male; 2=female)
    Year of Birth

dataformat2
    Ride ID
    Rideable type
    Started at
    Ended at
    Start station name
    Start station ID
    End station name
    End station ID
    Start latitude
    Start longitude
    End latitude
    End Longitude
    Member or casual ride


DOWNLOAD DATA
references

https://stackoverflow.com/questions/67284489/how-to-load-a-csv-file-from-zipped-folder-from-url-into-pandas-dataframe?noredirect=1&lq=1

https://stackoverflow.com/questions/3451111/unzipping-files-in-python

In [1]:
# Import dependencies
import zipfile
from urllib.request import urlopen
import shutil
import os
import pandas as pd


# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager


# import matplotlib.pyplot as plt
# import matplotlib.dates as mdates
# import mplcursors
# import pandas as pd
from datetime import datetime
from datetime import timezone
import numpy as np
# import os

import glob
import time
import requests
import bs4

In [2]:
#  # USED TO PULL DATA FROM WEBSITE - DO NOT NEED TO RERUN IF CSVs are in Resources directory
baseurl = 'https://s3.amazonaws.com/tripdata/'

In [3]:
#  # USED TO PULL DATA FROM WEBSITE - DO NOT NEED TO RERUN IF CSVs are in Resources directory
indexurl = 'https://s3.amazonaws.com/tripdata/index.html'

In [4]:
#  # USED TO PULL DATA FROM WEBSITE - DO NOT NEED TO RERUN IF CSVs are in Resources directory
# # this opens the test browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)


In [5]:
#  # USED TO PULL DATA FROM WEBSITE - DO NOT NEED TO RERUN IF CSVs are in Resources directory
# # scrape list of zip files from 'https://s3.amazonaws.com/tripdata/index.html'
browser.visit(indexurl)
time.sleep(5)
html = browser.html
tablelist = pd.read_html(html)
tabledf = tablelist[0]
tabledf.head()

,Name,Date Modified,Size,Type
0,201306-citibike-tripdata.zip,"Apr 30th 2018, 08:18:55 am",16.79 MB,ZIP file
1,201307-201402-citibike-tripdata.zip,"Jan 18th 2017, 04:23:25 pm",178.26 MB,ZIP file
2,201307-citibike-tripdata.zip,"Jan 18th 2017, 04:23:27 pm",27.07 MB,ZIP file
3,201308-citibike-tripdata.zip,"Jan 18th 2017, 04:23:27 pm",32.09 MB,ZIP file
4,201309-citibike-tripdata.zip,"Jan 18th 2017, 04:23:27 pm",33.16 MB,ZIP file


In [6]:
#  # USED TO PULL DATA FROM WEBSITE - DO NOT NEED TO RERUN IF CSVs are in Resources directory
# # select subset of data from pandas dataframe where JCzips.Name contains JC
JCzips = tabledf.loc[tabledf["Name"].str.contains('JC')]
JCzips

,Name,Date Modified,Size,Type
116,JC-201509-citibike-tripdata.csv.zip,"Apr 6th 2017, 04:01:34 pm",125 KB,ZIP file
117,JC-201510-citibike-tripdata.csv.zip,"Apr 6th 2017, 04:01:35 pm",357 KB,ZIP file
118,JC-201511-citibike-tripdata.csv.zip,"Apr 6th 2017, 04:01:37 pm",281 KB,ZIP file
119,JC-201512-citibike-tripdata.csv.zip,"Apr 6th 2017, 04:01:38 pm",222 KB,ZIP file
120,JC-201601-citibike-tripdata.csv.zip,"Apr 6th 2017, 04:01:38 pm",141 KB,ZIP file
...,...,...,...,...
199,JC-202208-citibike-tripdata.csv.zip,"Sep 8th 2022, 05:29:34 pm",3.38 MB,ZIP file
200,JC-202209-citibike-tripdata.csv.zip,"Oct 11th 2022, 11:55:32 am",3.38 MB,ZIP file
201,JC-202210-citibike-tripdata.csv.zip,"Nov 8th 2022, 05:00:25 pm",2.94 MB,ZIP file
202,JC-202211-citibike-tripdata.csv.zip,"Dec 5th 2022, 02:35:28 pm",2.30 MB,ZIP file


In [7]:
#  # USED TO PULL DATA FROM WEBSITE - DO NOT NEED TO RERUN IF CSVs are in Resources directory
# # create list from pandas dataframe column JCzips.Name
lJCzips = JCzips.Name.values.tolist()
print(lJCzips)


['JC-201509-citibike-tripdata.csv.zip', 'JC-201510-citibike-tripdata.csv.zip', 'JC-201511-citibike-tripdata.csv.zip', 'JC-201512-citibike-tripdata.csv.zip', 'JC-201601-citibike-tripdata.csv.zip', 'JC-201602-citibike-tripdata.csv.zip', 'JC-201603-citibike-tripdata.csv.zip', 'JC-201604-citibike-tripdata.csv.zip', 'JC-201605-citibike-tripdata.csv.zip', 'JC-201606-citibike-tripdata.csv.zip', 'JC-201607-citibike-tripdata.csv.zip', 'JC-201608-citibike-tripdata.csv.zip', 'JC-201609-citibike-tripdata.csv.zip', 'JC-201610-citibike-tripdata.csv.zip', 'JC-201611-citibike-tripdata.csv.zip', 'JC-201612-citibike-tripdata.csv.zip', 'JC-201701-citibike-tripdata.csv.zip', 'JC-201702-citibike-tripdata.csv.zip', 'JC-201703-citibike-tripdata.csv.zip', 'JC-201704-citibike-tripdata.csv.zip', 'JC-201705-citibike-tripdata.csv.zip', 'JC-201706-citibike-tripdata.csv.zip', 'JC-201707-citibike-tripdata.csv.zip', 'JC-201708 citibike-tripdata.csv.zip', 'JC-201709-citibike-tripdata.csv.zip', 'JC-201710-citibike-trip

In [8]:
# # need to loop through list and replace blanks with a dash  DO NOT USE - WILL NOT CAPTURE 201708 zip file
lJCzips2 = []

for z in lJCzips:
    zz = z.replace(" ","-")
    lJCzips2.append(zz)

print(lJCzips2)

['JC-201509-citibike-tripdata.csv.zip', 'JC-201510-citibike-tripdata.csv.zip', 'JC-201511-citibike-tripdata.csv.zip', 'JC-201512-citibike-tripdata.csv.zip', 'JC-201601-citibike-tripdata.csv.zip', 'JC-201602-citibike-tripdata.csv.zip', 'JC-201603-citibike-tripdata.csv.zip', 'JC-201604-citibike-tripdata.csv.zip', 'JC-201605-citibike-tripdata.csv.zip', 'JC-201606-citibike-tripdata.csv.zip', 'JC-201607-citibike-tripdata.csv.zip', 'JC-201608-citibike-tripdata.csv.zip', 'JC-201609-citibike-tripdata.csv.zip', 'JC-201610-citibike-tripdata.csv.zip', 'JC-201611-citibike-tripdata.csv.zip', 'JC-201612-citibike-tripdata.csv.zip', 'JC-201701-citibike-tripdata.csv.zip', 'JC-201702-citibike-tripdata.csv.zip', 'JC-201703-citibike-tripdata.csv.zip', 'JC-201704-citibike-tripdata.csv.zip', 'JC-201705-citibike-tripdata.csv.zip', 'JC-201706-citibike-tripdata.csv.zip', 'JC-201707-citibike-tripdata.csv.zip', 'JC-201708-citibike-tripdata.csv.zip', 'JC-201709-citibike-tripdata.csv.zip', 'JC-201710-citibike-trip

In [9]:
#  # USED TO PULL DATA FROM WEBSITE - DO NOT NEED TO RERUN IF CSVs are in Resources directory
# # need to loop through list and replace blanks with a dash  DO NOT USE - WILL NOT CAPTURE 201708 zip file
lJCzips3 = []

for z in lJCzips:
    zz = z.replace(" ","%20")
    lJCzips3.append(zz)

print(lJCzips3)

['JC-201509-citibike-tripdata.csv.zip', 'JC-201510-citibike-tripdata.csv.zip', 'JC-201511-citibike-tripdata.csv.zip', 'JC-201512-citibike-tripdata.csv.zip', 'JC-201601-citibike-tripdata.csv.zip', 'JC-201602-citibike-tripdata.csv.zip', 'JC-201603-citibike-tripdata.csv.zip', 'JC-201604-citibike-tripdata.csv.zip', 'JC-201605-citibike-tripdata.csv.zip', 'JC-201606-citibike-tripdata.csv.zip', 'JC-201607-citibike-tripdata.csv.zip', 'JC-201608-citibike-tripdata.csv.zip', 'JC-201609-citibike-tripdata.csv.zip', 'JC-201610-citibike-tripdata.csv.zip', 'JC-201611-citibike-tripdata.csv.zip', 'JC-201612-citibike-tripdata.csv.zip', 'JC-201701-citibike-tripdata.csv.zip', 'JC-201702-citibike-tripdata.csv.zip', 'JC-201703-citibike-tripdata.csv.zip', 'JC-201704-citibike-tripdata.csv.zip', 'JC-201705-citibike-tripdata.csv.zip', 'JC-201706-citibike-tripdata.csv.zip', 'JC-201707-citibike-tripdata.csv.zip', 'JC-201708%20citibike-tripdata.csv.zip', 'JC-201709-citibike-tripdata.csv.zip', 'JC-201710-citibike-tr

In [10]:
#  # USED TO PULL DATA FROM WEBSITE - DO NOT NEED TO RERUN IF CSVs are in Resources directory
#  # extracting zipfile from URL - works for everything!

for JC3 in lJCzips3:
        # searchindex = lJCzips2.index(JC2)
        # # print(searchindex)
        # filename = lJCzips[searchindex]
        # print(filename)   
        try:
            # zipfile_name = f'JC-{ym}-citibike-tripdata.csv.zip'
            # file_name = f'JC-{YYYYMM}-citibike-tripdata.csv'

            with urlopen(baseurl+JC3) as response, open(JC3, 'wb') as out_file:
                shutil.copyfileobj(response, out_file)

                # extracting required file from zipfile
                with zipfile.ZipFile(JC3) as zf:
                    zf.extractall('Resources')
            # deleting the zipfile from the directory
            os.remove(JC3)

            # loading data from the file
            # data = pd.read_csv(f'Resources/{file_name}')
        except:
            print(f"{JC3} not found")

In [11]:
#  # USED TO PULL DATA FROM WEBSITE - DO NOT NEED TO RERUN IF CSVs are in Resources directory
browser.quit()

In [12]:
# get list of csvs from file
allFiles = os.listdir("Resources")    
csvFiles = list(filter(lambda f: f.endswith('.csv'), allFiles))
csvFiles

['JC-201509-citibike-tripdata.csv',
 'JC-201510-citibike-tripdata.csv',
 'JC-201511-citibike-tripdata.csv',
 'JC-201512-citibike-tripdata.csv',
 'JC-201604-citibike-tripdata.csv',
 'JC-201605-citibike-tripdata.csv',
 'JC-201606-citibike-tripdata.csv',
 'JC-201607-citibike-tripdata.csv',
 'JC-201608-citibike-tripdata.csv',
 'JC-201609-citibike-tripdata.csv',
 'JC-20161-citibike-tripdata.csv',
 'JC-201610-citibike-tripdata.csv',
 'JC-201611-citibike-tripdata.csv',
 'JC-201612-citibike-tripdata.csv',
 'JC-20162-citibike-tripdata.csv',
 'JC-20163-citibike-tripdata.csv',
 'JC-201701-citibike-tripdata.csv',
 'JC-201702-citibike-tripdata.csv',
 'JC-201703-citibike-tripdata.csv',
 'JC-201704-citibike-tripdata.csv',
 'JC-201705-citibike-tripdata.csv',
 'JC-201706-citibike-tripdata.csv',
 'JC-201707-citibike-tripdata.csv',
 'JC-201708 citibike-tripdata.csv',
 'JC-201709-citibike-tripdata.csv',
 'JC-201710-citibike-tripdata.csv',
 'JC-201711-citibike-tripdata.csv',
 'JC-201712-citibike-tripdata.c

In [13]:
# loop through csv files and write to lists
ldata1 = [] #list for dfs with data format 1
ldata2 = [] #list for dfs with data format 2
ldata3 = [] #list to catch unmatched dfs
for csvFile in csvFiles:
    caticorn = pd.read_csv(f'Resources/{csvFile}')
    if caticorn.columns[0] == 'Trip Duration' or caticorn.columns[0] == 'tripduration':
        ldata1.append(caticorn)
    elif caticorn.columns[0] == 'ride_id':
        ldata2.append(caticorn)
    else:
        ldata3.append(caticorn)

In [14]:
#check number of columns in ldata1
# print(len(ldata1[0].columns))
lspinch = []
for spinch in ldata1:
    if len(spinch.columns) not in lspinch:
        lspinch.append(len(spinch.columns))

for ldata1_n_cols in lspinch:
    print(ldata1_n_cols)

# reference: check for changes in column names between dataframes for ldata1
print(len(ldata1))
lturnip = []
# lturnipseed = []
for turnip in ldata1:
    for turnipseed in range(0,ldata1_n_cols):
        if turnip.columns[turnipseed] not in lturnip:
            lturnip.append(turnip.columns[turnipseed])
        
for t in lturnip:
    print (t)

15
65
Trip Duration
Start Time
Stop Time
Start Station ID
Start Station Name
Start Station Latitude
Start Station Longitude
End Station ID
End Station Name
End Station Latitude
End Station Longitude
Bike ID
User Type
Birth Year
Gender
tripduration
starttime
stoptime
start station id
start station name
start station latitude
start station longitude
end station id
end station name
end station latitude
end station longitude
bikeid
usertype
birth year
gender


In [15]:
print(ldata1_n_cols)

15


In [16]:
ldata1_new_colnames = ['tripduration','started_at','ended_at', 'start_station_id','start_station_name','start_lat','start_lng','end_station_id','end_station_name','end_lat','end_lng','bikeid','member_casual','birthyear','gender']

In [17]:
# create clean ldata1 list:  update column names to match ldata2 columns (where applicable)
# parsnip and derivatives are for-loop variables only

ldata1clean = []
for parsnip in ldata1:
    parsnip.columns = ldata1_new_colnames
    parsnip['ride_id'] = [f"{parsnip['bikeid'][ele]} {parsnip['started_at'][ele]}" for ele in parsnip.index]
    parsnip['rideable_type'] = parsnip['bikeid']
    parsnip['started_at'] = parsnip['started_at'].astype('datetime64')
    parsnip['ended_at'] = parsnip['ended_at'].astype('datetime64')
    parsnip['rideable_type'] = parsnip['rideable_type'].astype('str')
    parsnip['start_station_id'] = parsnip['start_station_id'].astype('str')
    parsnip['end_station_id'] = parsnip['end_station_id'].astype('str')   
    parsnippy = parsnip[['ride_id','rideable_type','started_at','ended_at','start_station_name','start_station_id','end_station_name','end_station_id','start_lat','start_lng','end_lat','end_lng','member_casual','tripduration']]
    ldata1clean.append(parsnippy)
        


In [18]:
# reference: check for changes in column names between dataframes for ldata1clean
#  spinch, turnip and their derivatives are for-loop variables only
lspinch4 = []
for spinch4 in ldata1clean:
    if len(spinch4.columns) not in lspinch4:
        lspinch4.append(len(spinch4.columns))

for ldata1clean_n_cols in lspinch4:
    print(ldata1clean_n_cols)

lturnip4 = []
# lturnipseed = []
for turnip4 in ldata1clean:
    for turnipseed4 in range(0,ldata1clean_n_cols):
        if turnip4.columns[turnipseed4] not in lturnip4:
            lturnip4.append(turnip4.columns[turnipseed4])
        
for t4 in lturnip4:
    print (t4)

14
ride_id
rideable_type
started_at
ended_at
start_station_name
start_station_id
end_station_name
end_station_id
start_lat
start_lng
end_lat
end_lng
member_casual
tripduration


In [19]:
ldata1clean[0].head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,tripduration
0,24722 2015-09-21 14:53:16,24722,2015-09-21 14:53:16,2015-09-21 14:54:17,City Hall,3185,City Hall,3185,40.717732,-74.043845,40.717732,-74.043845,Subscriber,61
1,24388 2015-09-21 14:55:59,24388,2015-09-21 14:55:59,2015-09-21 15:00:50,Exchange Place,3183,Warren St,3187,40.716247,-74.033459,40.721124,-74.038051,Customer,290
2,24442 2015-09-21 14:56:14,24442,2015-09-21 14:56:14,2015-09-21 15:09:21,Exchange Place,3183,Exchange Place,3183,40.716247,-74.033459,40.716247,-74.033459,Subscriber,786
3,24678 2015-09-21 14:58:35,24678,2015-09-21 14:58:35,2015-09-21 15:06:32,Hamilton Park,3203,Hamilton Park,3203,40.727596,-74.044247,40.727596,-74.044247,Subscriber,477
4,24574 2015-09-21 14:59:55,24574,2015-09-21 14:59:55,2015-09-21 15:07:27,Hamilton Park,3203,Grove St PATH,3186,40.727596,-74.044247,40.719586,-74.043117,Subscriber,451


In [20]:
ldata1clean[0].dtypes

ride_id                       object
rideable_type                 object
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name            object
start_station_id              object
end_station_name              object
end_station_id                object
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual                 object
tripduration                   int64
dtype: object

In [21]:
##  DO NOT USE THIS CODE - FOR CONCEPT ONLY - check out how to change column types, calculate trip duration
# ldata1clean0df = ldata1clean[0].copy()
# ldata1clean0df['started_at'] = ldata1clean0df['started_at'].astype('datetime64')
# ldata1clean0df['ended_at'] = ldata1clean0df['ended_at'].astype('datetime64')
# ldata1clean0df['duration'] = (ldata1clean0df['ended_at']- ldata1clean0df['started_at'])/1000000000
# ldata1clean0df['duration'] = ldata1clean0df['duration'].astype('int64')
# ldata1clean0df['rideable_type'] = ldata1clean0df['rideable_type'].astype('str')
# ldata1clean0df['start_station_id'] = ldata1clean0df['start_station_id'].astype('str')
# ldata1clean0df['end_station_id'] = ldata1clean0df['end_station_id'].astype('str')   
# ldata1clean0df.head()


In [22]:
# ##  DO NOT USE THIS CODE - FOR CONCEPT ONLY - check out how to change column types, calculate trip duration - check that changes were applied to df
# ldata1clean0df.dtypes

In [23]:
#check number of columns in ldata2
#  spinch, turnip and their derivatives are for-loop variables only
lspinch2 = []
for spinch2 in ldata2:
    if len(spinch2.columns) not in lspinch2:
        lspinch2.append(len(spinch2.columns))

for ldata2_n_cols in lspinch2:
    print(ldata2_n_cols)

# reference: check for changes in column names between dataframes for ldata2
print(len(ldata2))
lturnip2 = []
# lturnipseed2 = []
for turnip2 in ldata2:
    for turnipseed2 in range(0,ldata2_n_cols):
        if turnip2.columns[turnipseed2] not in lturnip2:
            lturnip2.append(turnip2.columns[turnipseed2])
        
for t2 in lturnip2:
    print (t2)

13
23
ride_id
rideable_type
started_at
ended_at
start_station_name
start_station_id
end_station_name
end_station_id
start_lat
start_lng
end_lat
end_lng
member_casual


In [24]:
# create clean ldata2 list:  add column to match ldata1 tripduration
# parsnip and derivatives are for-loop variables only

ldata2clean = []
for parsnip8 in ldata2:
    parsnip8['started_at'] = parsnip8['started_at'].astype('datetime64')
    parsnip8['ended_at'] = parsnip8['ended_at'].astype('datetime64')
    parsnip8['tripduration'] = [(parsnip8['ended_at'][row]-parsnip8['started_at'][row])/1000000000 for row in parsnip8.index]
    parsnip8['tripduration'] = parsnip8['tripduration'].astype('int64')
    parsnippy8 = parsnip8[['ride_id','rideable_type','started_at','ended_at','start_station_name','start_station_id','end_station_name','end_station_id','start_lat','start_lng','end_lat','end_lng','member_casual','tripduration']]
    ldata2clean.append(parsnippy8)

In [25]:
# reference: check for changes in column names between dataframes for ldata2clean
#  spinch, turnip and their derivatives are for-loop variables only
lspinch4 = []
for spinch4 in ldata2clean:
    if len(spinch4.columns) not in lspinch4:
        lspinch4.append(len(spinch4.columns))

for ldata1clean_n_cols in lspinch4:
    print(ldata1clean_n_cols)

lturnip4 = []
# lturnipseed = []
for turnip4 in ldata2clean:
    for turnipseed4 in range(0,ldata1clean_n_cols):
        if turnip4.columns[turnipseed4] not in lturnip4:
            lturnip4.append(turnip4.columns[turnipseed4])
        
for t4 in lturnip4:
    print (t4)

14
ride_id
rideable_type
started_at
ended_at
start_station_name
start_station_id
end_station_name
end_station_id
start_lat
start_lng
end_lat
end_lng
member_casual
tripduration


In [26]:
ldata2clean[0].dtypes

ride_id                       object
rideable_type                 object
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name            object
start_station_id              object
end_station_name              object
end_station_id                object
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual                 object
tripduration                   int64
dtype: object

In [27]:
ldata2clean[0].head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,tripduration
0,121DD7DD23CB1335,docked_bike,2021-02-03 23:11:28,2021-02-03 23:18:28,Hoboken Ave at Monmouth St,JC105,Christ Hospital,JC034,40.735208,-74.046964,40.734786,-74.050444,member,420
1,FD73FB85F008349D,docked_bike,2021-02-27 16:34:05,2021-02-27 16:56:40,Newport Pkwy,JC008,Marin Light Rail,JC013,40.728744,-74.032108,40.714584,-74.042817,member,1355
2,39F9E6663CB5FDF6,docked_bike,2021-02-26 23:16:04,2021-02-26 23:22:25,Journal Square,JC103,Baldwin at Montgomery,JC020,40.733670,-74.062500,40.723659,-74.064194,member,381
3,A64745CB0792EC6F,docked_bike,2021-02-24 16:51:50,2021-02-24 17:16:09,Hoboken Ave at Monmouth St,JC105,Hoboken Ave at Monmouth St,JC105,40.735208,-74.046963,40.735208,-74.046964,casual,1459
4,75CC76EB9543764A,docked_bike,2021-02-24 20:44:16,2021-02-24 20:44:46,Hoboken Ave at Monmouth St,JC105,Hoboken Ave at Monmouth St,JC105,40.735208,-74.046963,40.735208,-74.046964,member,30


In [28]:
dfdata1clean = pd.concat(ldata1clean)
len(dfdata1clean)

1702660

In [29]:
dfdata2clean = pd.concat(ldata2clean)
len(dfdata2clean)

1528304

In [30]:
joinedlist = ldata1clean + ldata2clean
len(joinedlist)

88

In [31]:
dfdataall = pd.concat(joinedlist)

In [32]:
len(dfdataall)

3230964

In [34]:
# merged csv is too large for github - save csv outside of local repository
# dfdataall.to_csv('Resources/JC_201509_202212_citibike_tripdata.csv')

Unneeded code but keep for future reference

In [33]:
# reference: create YYYYMM list
# not needed but cool loop: create YYYYMM list 
# lyyyymm  = []
# for y in range (2015,2023):
#     for m in range (1,13):
#         if m <10:
#             lyyyymm.append(f'{y}0{m}')
#         else:
#             lyyyymm.append(f'{y}{m}')

# print(lyyyymm)
# len(lyyyymm)
